In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import glob
import re
import numpy as np
from datetime import datetime
from tqdm import tqdm

import traceback

In [ ]:
file_paths = glob.glob('data/*.html')
list_of_dfs = []
for file_path in file_paths:
    print(f"Processing file: {file_path}")
    with open(file_path, 'r', encoding='utf-8') as f:
        html_content = f.read()

    soup = BeautifulSoup(html_content, 'html.parser')
    header = soup.find('div', class_='particle-table-header')

    if not header:
        print(f"  --> Warning: Could not find a header in {file_path}. Skipping this file.")
        continue

    columns = [cell.text.strip() for cell in header.find_all('aw-header-cell')]

    rows = []
    for row in soup.find_all('div', class_='particle-table-row'):
        cells = row.find_all('ess-cell')
        if len(cells) == len(columns):
            row_data = [cell.text.strip() for cell in cells]
            rows.append(row_data)
        else:
            print(f"  --> Warning: Mismatch between column count and cell count in a row within {file_path}.")

    df_current_file = pd.DataFrame(rows, columns=columns)

    date_str_match = re.search(r'keywords_(\d{2}-\d{2}-\d{2})\.html', file_path)
    if date_str_match:
        date_str = date_str_match.group(1)
        year_part = date_str.split('-')[0]
        full_year_str = '20' + year_part
        corrected_date_str = f"{full_year_str}-{date_str.split('-')[1]}-{date_str.split('-')[2]}"
        df_current_file['date'] = pd.to_datetime(corrected_date_str)
    else:
        df_current_file['date'] = pd.NaT


    list_of_dfs.append(df_current_file)

if list_of_dfs:
    df = pd.concat(list_of_dfs, ignore_index=True)
else:
    print("No valid HTML files were found or processed. The final DataFrame is empty.")
    df = pd.DataFrame()

def to_snake_case(column_name):
    s = re.sub(r'[\s\n]+', ' ', column_name).strip()
    s = s.lower()
    s = re.sub(r'[().]', '', s)
    s = s.replace(' ', '_')
    s = s.replace('top_of_page_', '')
    return s

original_columns = df.columns
new_columns_map = {col: to_snake_case(col) for col in original_columns}
df.rename(columns=new_columns_map, inplace=True)

df = df.drop(columns=['account_status', 'ad_impression_share'])

df['competition'] = df['competition'].map({'Low': 1, 'Medium': 2, 'High': 3})
df['keyword'] = df['keyword'].apply(lambda x: re.sub(r'[\s\n]+', ' ', x).strip())

df['avg_monthly_searches'] = df['avg_monthly_searches'].str.split(' –').str[0]
df['avg_monthly_searches'] = df['avg_monthly_searches'].str.replace('K', '000', regex=False)
df['avg_monthly_searches'] = df['avg_monthly_searches'].str.replace('M', '000000', regex=False)
df['avg_monthly_searches'] = pd.to_numeric(df['avg_monthly_searches'], errors='coerce')

for col in ['bid_low_range', 'bid_high_range']:
    df[col] = df[col].str.replace('€', '', regex=False)
    df[col] = df[col].str.replace('—', '', regex=False).str.strip()
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['avg_bid'] = df[['bid_high_range', 'bid_low_range']].mean(axis=1)

for col in ['three_month_change', 'yoy_change']:
    df[col] = df[col].str.replace('%', '', regex=False)
    df[col] = df[col].str.replace(',', '', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] /= 100

df = (df.sort_values(by='date')
        .drop_duplicates(subset='keyword', keep='last'))

df['keyword_len'] = df['keyword'].apply(lambda x: len(x))
df['keyword_len'] = (df['keyword_len'] - df['keyword_len'].min()) / (df['keyword_len'].max() - df['keyword_len'].min())

df['avg_monthly_searches_norm'] = np.log1p(df['avg_monthly_searches'])
df['avg_monthly_searches_norm'] = (df['avg_monthly_searches_norm'] - df['avg_monthly_searches_norm'].min()) / (df['avg_monthly_searches_norm'].max() - df['avg_monthly_searches_norm'].min())

df['score'] = df['avg_monthly_searches_norm'] - df['keyword_len']
df['relative_volume'] = np.nan

df = df.sort_values(by='score', ascending=False)
df

In [ ]:
import json
import numpy as np
from datetime import datetime, timezone
from time import sleep
from urllib.parse import quote, urlparse, parse_qs

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
# Fundamental set up
def setup_driver(url="https://trends.google.com/trends/explore"):
    options = webdriver.ChromeOptions()
    options.set_capability("goog:loggingPrefs", {"performance": "INFO"})
    options.add_argument("--no-sandbox")
    # options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-extensions")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)
    driver.execute_cdp_cmd("Network.enable", {})
    driver.get(url)
    return driver


def parse_json_body(entry):
    body = entry.get("body", "{}")
    if body.startswith(")]}'"):
        body = body.split("\n", 1)[1]
    return json.loads(body)


def get_multiline_data(driver, log_json):
    request_id = log_json["params"]["requestId"]
    response_body = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})
    parsed = parse_json_body(response_body)

    timeline = parsed["default"]["timelineData"]
    dates = np.array([
        datetime.fromtimestamp(int(t["time"]), tz=timezone.utc).date()
        for t in timeline
    ])

    values_list = [t.get("value", []) for t in timeline]
    values = np.array(values_list, dtype=float)
    if values.ndim == 1:
        values = values.reshape(-1, 1) # Ensure values is 2D

    maximums = np.max(values, axis=0)

    return maximums, dates, values


def get_related_keywords(driver, log_json):
    full_url = log_json["params"]["response"]["url"]
    query_params = parse_qs(urlparse(full_url).query)
    query_req = json.loads(query_params['req'][0])
    keyword = query_req['restriction']['complexKeywordsRestriction']['keyword'][0]['value']

    request_id = log_json["params"]["requestId"]
    response_body = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})
    parsed = parse_json_body(response_body)
    related_keywords = parsed["default"]["rankedList"]
    
    query_value_dict = {}
    for keyword_dict in related_keywords[0]['rankedKeyword']:
        query_value_dict[keyword_dict['query']] = keyword_dict['value']

    return keyword, query_value_dict


def get_url(driver, batch_keywords):
    while driver.get_log("performance"): # clear stray logs
        pass
    encoded = [quote(kw) for kw in batch_keywords]
    url = f"https://trends.google.com/trends/explore?q={','.join(encoded)}&hl=en"
    driver.get(url)
    WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.ID, "TIMESERIES")))


def parse_logs(driver, batch_keywords):
    current_batch_data = {
        "keywords": batch_keywords,
        "related_keywords": {kw:{} for kw in batch_keywords}
    }

    logs = driver.get_log("performance")
    all_logs = logs
    while logs:
        sleep(1)
        logs = driver.get_log("performance")
        all_logs.extend(logs)

    for log in all_logs:
        log_json = json.loads(log["message"])["message"]
        if log_json.get("method") == "Network.responseReceived":
            response_url = log_json["params"]["response"]["url"]
            if "api/widgetdata/multiline" in response_url:
                maximums, dates, values = get_multiline_data(driver, log_json)
                current_batch_data['dates'] = dates
                current_batch_data['values'] = values
                current_batch_data['maximums'] = maximums
            elif "api/widgetdata/relatedsearches" in response_url:
                keyword, related = get_related_keywords(driver, log_json)
                current_batch_data['related_keywords'][keyword] = related
    
    return current_batch_data


In [ ]:
# Latest
def subset_batch(batch, keywords):
    indices = [batch["keywords"].index(kw) for kw in keywords if kw in batch["keywords"]]
    values = batch["values"][:, indices]
    maximums = [batch["maximums"][i] for i in indices]
    related_keywords = {kw: batch["related_keywords"].get(kw, {}) for kw in keywords}
    
    return {
        "dates": batch["dates"],
        "keywords": keywords,
        "values": values,
        "maximums": maximums,
        "related_keywords": related_keywords,
    }


def detect_outlier(values, keywords, bridge_rank=3, min_mean_volume=2):
    means = np.mean(values, axis=0)
    nonzero = means[means > min_mean_volume]
    outlier = keywords[np.argmax(means)]
    if len(nonzero) <= min(len(keywords), bridge_rank) and len(keywords):
        return True, outlier
    return False, None


def detect_low_outliers(values, keywords, bridge_keyword, min_mean_volume=2):
    means = np.mean(values, axis=0)
    low_outliers_idx = np.where(means < min_mean_volume)[0]
    low_outliers = [keywords[i] for i in low_outliers_idx]
    high_volumes = [kw for kw in keywords if kw not in low_outliers]

    if low_outliers:
        if bridge_keyword in low_outliers:
            return high_volumes
        

    return None


def choose_bridge_keyword(batch_keywords, values, bridge_rank=3, min_mean_volume=2):
    means = np.mean(values, axis=0)
    kw_means = sorted(zip(batch_keywords, means), key=lambda x: x[1], reverse=True)
    kw_means = [kw_mean for kw_mean in kw_means if kw_mean[1] > min_mean_volume]
    if len(kw_means) >= bridge_rank:
        return kw_means[bridge_rank - 1][0]
    else:
        return kw_means[-1][0]


def scrape_batches(driver, sorted_keywords, min_mean_volume=2, max_retries=3, bridge_rank=3):
    batches, retry_count, bridge_keyword = [], 0, None
    
    keywords_to_process = list(sorted_keywords)
    while keywords_to_process:
        if bridge_keyword:
            new_keywords = keywords_to_process[:4]
            batch_keywords = [bridge_keyword] + new_keywords
        else:
            batch_keywords = new_keywords = keywords_to_process[:5]
        batch_keywords = list(set(batch_keywords))

        try:
            get_url(driver, batch_keywords)
            current_batch_dict = parse_logs(driver, batch_keywords)
        except Exception:
            retry_count += 1
            if retry_count > max_retries:
                print(f"Exceeded max retries ({max_retries})")
                return batches, []
            continue
        if 'values' not in current_batch_dict:
            retry_count += 1
            if retry_count > max_retries:
                print(f"Exceeded max retries ({max_retries})")
                batches, []
            continue

        values = np.array(current_batch_dict["values"])
        retry_count = 0
        keywords_to_process = keywords_to_process[len(new_keywords):]

        outlier_kw = detect_outlier(values,
                                    batch_keywords,
                                    bridge_keyword,
                                    min_mean_volume=min_mean_volume)
        if outlier_kw:
            if not bridge_keyword or bridge_keyword == outlier_kw:
                bridge_keyword = choose_bridge_keyword(batch_keywords,
                                                       values,
                                                       bridge_rank=bridge_rank,
                                                       min_mean_volume=min_mean_volume)

            distilled_batch = subset_batch(current_batch_dict, [bridge_keyword, outlier_kw])
            batches.append(distilled_batch)

            leftover_keywords = [kw for kw in new_keywords if kw != outlier_kw]
            keywords_to_process = leftover_keywords + keywords_to_process
            print(f"{bridge_keyword}")
            print(f"Outlier: {outlier_kw} -- {current_batch_dict['keywords']}")
        else:
            batches.append(current_batch_dict)
            print(f"{bridge_keyword}")
            print(f"{current_batch_dict['keywords']}")
            bridge_keyword = choose_bridge_keyword(batch_keywords,
                                                   values,
                                                   bridge_rank=bridge_rank,
                                                   min_mean_volume=min_mean_volume)

    return batches, keywords_to_process


def join_batches(batches):
    if not batches:
        return np.array([]), []

    concatenated_keywords = list(batches[0]["keywords"])
    concatenated_values = np.array(batches[0]["values"])
    if concatenated_values.ndim == 1:
        concatenated_values = concatenated_values[:, np.newaxis]

    i = 1
    while i < len(batches):
        current = batches[i]
        current_values = np.array(current["values"])
        if current_values.ndim == 1:
            current_values = current_values[:, np.newaxis]

        bridge_keywords_set = set(concatenated_keywords) & set(current["keywords"])
        bridge_keyword = list(bridge_keywords_set)[0]
        prev_idx = concatenated_keywords.index(bridge_keyword)
        current_idx = current["keywords"].index(bridge_keyword)

        mean_prev = concatenated_values[:, prev_idx].mean()
        mean_current = current_values[:, current_idx].mean()
        scale = mean_prev / np.maximum(mean_current, 1e-9)

        scaled = current_values * scale
        keep_cols = [j for j, kw in enumerate(current["keywords"]) if kw not in concatenated_keywords]
        concatenated_values = np.column_stack((concatenated_values, scaled[:, keep_cols]))
        concatenated_keywords.extend([current["keywords"][j] for j in keep_cols])
        
        i += 1

    return concatenated_values, concatenated_keywords


def resort_df(df, trends_df):
    new_relative_volume = pd.DataFrame(trends_df.mean()).reset_index().rename(columns={'index':'keyword', 0:'relative_volume'})
    df = df.drop(columns='relative_volume', errors='ignore')
    df = df.merge(new_relative_volume, on='keyword', how='left')
    return df.sort_values(by=['relative_volume', 'score'], ascending=[False, False])


def get_trends_data(driver, initial_df, min_mean_volume=2, bridge_rank=3):
    keywords_to_process = initial_df['keyword'].tolist()
    
    batches = scrape_batches(driver,
                             keywords_to_process,
                             min_mean_volume=min_mean_volume,
                             bridge_rank=bridge_rank)
    
    joined_values, joined_keywords = join_batches(batches)
    trends_df = pd.DataFrame(joined_values, columns=joined_keywords)
    final_df = resort_df(initial_df, trends_df)
    
    related_keywords = {}
    for batch in batches:
        related_keywords.update(batch.get("related_keywords", {}))
    
    return final_df, trends_df, related_keywords


In [ ]:
# df = pd.read_csv('data/planner_keywords_sorted.csv')
# df['date'] = pd.to_datetime(df['date'])

# driver = setup_driver()
# input("Press Enter after login/CAPTCHA if needed...")

BRIDGE_RANK = 3
MIN_MEAN_VOLUME = 3
keywords_to_process = df['keyword'].unique().tolist()[-15:]


stored_and_sorted = set()
max_iterations = len(df) // 10

all_batches = []
for i in range(max_iterations):
    print(f"\nIteration {i+1} - Keywords remaining: {len(keywords_to_process)}")
    newly_scraped_batches, remaining_keywords = scrape_batches(driver,
                                                               keywords_to_process,
                                                               bridge_rank=BRIDGE_RANK,
                                                               min_mean_volume=MIN_MEAN_VOLUME)
    all_batches.extend(newly_scraped_batches)

    joined_values, joined_keywords = join_batches(all_batches)
    trends_df = pd.DataFrame(joined_values, columns=joined_keywords)
    df = resort_df(df, trends_df)

    if remaining_keywords:
        next_new_keyword = remaining_keywords[0]
        resume_index = df.index[df['keyword'] == next_new_keyword][0]
        start_index = max(0, resume_index - 1)
        keywords_to_process = [k for k in df['keyword'].iloc[start_index:].tolist() if k not in stored_and_sorted]
    else:
        break

joined_values, joined_keywords = join_batches(all_batches)
trends_df = pd.DataFrame(joined_values, columns=joined_keywords)
df = resort_df(df, trends_df)
df

# # Final pass
# final_batches = []
# print("\nFinal iteration")    
# final_batches, remaining_keywords = scrape_batches(
#     driver, 
#     df['keyword'].to_list(), 
#     bridge_rank=BRIDGE_RANK, 
#     min_mean_volume=MIN_MEAN_VOLUME,
# )
# if remaining_keywords:
#     print("Final pass was incomplete:")
#     print(remaining_keywords)

# joined_values, joined_keywords = join_batches(final_batches)
# trends_df = pd.DataFrame(joined_values, columns=joined_keywords)
# df = resort_df(df, trends_df)
# df['date'] = df['date'].dt.date
# df.to_csv('data/planner_keywords_sorted.csv', index=False)

# related_keywords = {}
# for batch in all_batches:
#     related_keywords.update(batch.get("related_keywords", {}))

# driver.quit()

In [ ]:
def create_unified_keyword_series_refined(trends_df, related_keywords):
    sorted_scalars = trends_df.mean(axis=0)

    related_data = []
    for parent_kw, rel_dict in related_keywords.items():
        if parent_kw not in sorted_scalars:
            continue
        parent_mean = sorted_scalars[parent_kw]
        for rel_kw, score in rel_dict.items():
            scaled_val = (score / 100) * parent_mean
            related_data.append((rel_kw, scaled_val))

    if related_data:
        related_df = pd.DataFrame(related_data, columns=['keyword', 'score'])
        related_scalars = related_df.groupby('keyword')['score'].mean()
    else:
        related_scalars = pd.Series(dtype=float)

    all_scalars = (
        pd.concat([sorted_scalars, related_scalars])
        .groupby(level=0).mean()
        .sort_values(ascending=False)
    )

    return all_scalars


In [ ]:
# all_keywords = create_unified_keyword_series_refined(trends_df, related_keywords)
# all_keywords = all_keywords[all_keywords.index.isin([kw for kw in all_keywords.index if kw not in STOP_WORDS])]
# all_keywords = all_keywords[all_keywords.index.isin([kw for kw in all_keywords.index if kw not in STOP_WORDS])]
# all_keywords = all_keywords.reset_index()
# all_keywords.columns = ['keyword', 'estimated_relative_volume']
# all_keywords['keyword_len'] = all_keywords['keyword'].apply(lambda x: len(x))
# all_keywords['keyword_len'] = (all_keywords['keyword_len'] - all_keywords['keyword_len'].min()) / (all_keywords['keyword_len'].max() - all_keywords['keyword_len'].min())

# all_keywords['avg_monthly_searches_norm'] = np.log1p(all_keywords['estimated_relative_volume'])
# all_keywords['avg_monthly_searches_norm'] = (all_keywords['avg_monthly_searches_norm'] - all_keywords['avg_monthly_searches_norm'].min()) / (all_keywords['avg_monthly_searches_norm'].max() - all_keywords['avg_monthly_searches_norm'].min())

# all_keywords['score'] = all_keywords['avg_monthly_searches_norm'] - all_keywords['keyword_len']

# df = all_keywords.copy()

# driver.quit()

In [ ]:
all_keywords = pd.read_csv('data/all_keywords.csv')
df = pd.read_csv('data/planner_keywords_sorted.csv')
df['date'] = pd.to_datetime(df['date'])


all_keywords = create_unified_keyword_series_refined(trends_df, related_keywords)
all_keywords = all_keywords[all_keywords.index.isin([kw for kw in all_keywords.index if kw not in STOP_WORDS])]
all_keywords = all_keywords.reset_index()
all_keywords.columns = ['keyword', 'estimated_relative_volume']


if input('save? (y/n)').lower().strip() == 'y':
    df = resort_df(df, trends_df)
    df['date'] = df['date'].dt.date
    df.to_csv('data/planner_keywords_sorted.csv', index=False)
    all_keywords.to_csv('data/all_keywords.csv', index=False)